# TP 1 Individual

## Alumno: Scordamaglia Ezequiel

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet, ElasticNetCV, LogisticRegression, LogisticRegressionCV, SGDClassifier, SGDRegressor
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
df = pd.read_csv('data\PS_20174392719_1491204439457_log.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [3]:
# Elimino la columna isFlaggedFraud, ya que mi targuet es isFraud
df.drop(columns=['isFlaggedFraud'], axis=1, inplace=True)
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0


In [4]:
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud'],
      dtype='object')

In [5]:
df.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
dtype: object

In [6]:
target = np.array(df['isFraud'])
df = df.drop('isFraud', axis = 1)
X_train, X_test, Y_train, Y_test = train_test_split(df, target, test_size = 0.25, random_state = 23)

In [7]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(4771965, 9)
(1590655, 9)
(4771965,)
(1590655,)


In [8]:
X_train.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
575698,25,PAYMENT,6502.78,C1746430570,39732.0,33229.22,M2010531827,0.00,0.00
260323,14,PAYMENT,8900.78,C809260502,17054.0,8153.22,M1315397035,0.00,0.00
5809937,401,TRANSFER,7633670.36,C1900703280,0.0,0.00,C670177943,22008830.12,29642500.48
4705099,331,CASH_IN,347554.14,C59641128,902199.6,1249753.75,C353885887,887679.11,540124.97
409266,18,CASH_OUT,77819.47,C1554236863,0.0,0.00,C1018210791,157879.64,85858.81


# Preprocessing

In [9]:
class SliceTransform():
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['nameOrig_type'] = X['nameOrig'].str.slice(stop=1)
        X['nameOrig_num'] = X['nameOrig'].str.slice(start=1)
        return X.drop('nameOrig', axis=1)

class DropTransform():
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X.drop(columns=['type'], axis=1, inplace=True)
        X = pd.get_dummies(X, columns=['nameOrig_type'], dtype=int)
        return X

class Extractor(BaseEstimator, TransformerMixin):
    def __init__(self, start=0, stop=None):
        self.start = start
        self.stop = stop
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.str.slice(start=self.start, stop=self.stop)

In [ ]:
pipeline = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ('slice', SliceTransform()),
    ('drop', DropTransform())
])

In [ ]:
pipeline.fit(X_train)

In [10]:
preprocessor = ColumnTransformer(transformers=[
    ('type_OHE', OneHotEncoder(dtype=int), ['type']),
    ('nameOrig_type', Extractor(stop=1), ['nameOrig']),
    ('nameOrig_num', Extractor(start=1), ['nameOrig']),
    ('nameDest_type', Extractor(stop=1), ['nameDest']),
    ('nameDest_num', Extractor(start=1), ['nameDest']),
    ('nameDest_type_OHE', OneHotEncoder(dtype=int), ['nameDest_type']),
    ('nameOrig_type_OHE', OneHotEncoder(dtype=int), ['nameOrig_type']),
])

In [16]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [ ]:
pipeline.fit(X_train, Y_train)

## Preprocesar el set de Test

In [ ]:
X_test = pd.get_dummies(X_test, columns=['type'], dtype=int)
X_test['nameOrig_type'] = X_test['nameOrig'].str.slice(stop=1)
X_test['nameOrig_num'] = X_test['nameOrig'].str.slice(start=1)
X_test['nameDest_type'] = X_test['nameDest'].str.slice(stop=1)
X_test['nameDest_num'] = X_test['nameDest'].str.slice(start=1)
X_test.drop('nameOrig', axis=1, inplace=True)
X_test.drop('nameDest', axis=1, inplace=True)
X_test = pd.get_dummies(X_test, columns=['nameOrig_type', 'nameDest_type'], dtype=int)

## LogisticRegressionCV

In [ ]:
logr = LogisticRegressionCV(cv=10, scoring='roc_auc', n_jobs=-1, penalty="l2")
logr.fit(X_train,Y_train)
logr_y_hat = logr.predict(X_test)